<a href="https://colab.research.google.com/github/zaraprvz/Diabetes-Prediction-Capstone-Project/blob/main/B2B_Courier_Charges_Accuracy_Analysis_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#unzip file
!!unzip "/content/b2b (2).zip"

['Archive:  /content/b2b (2).zip',
 '  inflating: Courier Company - Rates.csv  ',
 '  inflating: __MACOSX/._Courier Company - Rates.csv  ',
 '  inflating: Invoice.csv             ',
 '  inflating: __MACOSX/._Invoice.csv  ',
 '  inflating: Order Report.csv        ',
 '  inflating: __MACOSX/._Order Report.csv  ',
 '  inflating: pincodes.csv            ',
 '  inflating: __MACOSX/._pincodes.csv  ',
 '  inflating: SKU Master.csv          ',
 '  inflating: __MACOSX/._SKU Master.csv  ']

In [4]:
#Import libraries
import pandas as pd

order_report = pd.read_csv('Order Report.csv')
sku_master = pd.read_csv('SKU Master.csv')
pincode_mapping = pd.read_csv('pincodes.csv')
courier_invoice = pd.read_csv('Invoice.csv')
courier_company_rates = pd.read_csv('Courier Company - Rates.csv')

print(order_report.head())
print(sku_master.head())
print(pincode_mapping.head())
print(courier_invoice.head())
print(courier_company_rates.head())


   ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN
             SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN
   Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             121003            486886    d         NaN         NaN
2   

In [5]:
#Check if any data contain missing values
print("\nMissing values in Website Order Report")
print(order_report.isnull().sum())
print("\nMissing values in SKU Master")
print(sku_master.isnull().sum())
print("\nMissing values in Pincode Mapping")
print(pincode_mapping.isnull().sum())
print("\nMissing values in Courier Invoice")
print(courier_invoice.isnull().sum())
print("\nMissing values in Courier Company Rates")
print(courier_company_rates.isnull().sum())


Missing values in Website Order Report
ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64

Missing values in SKU Master
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64

Missing values in Pincode Mapping
Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64

Missing values in Courier Invoice
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing values in Courier Company Rates
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed         0
rt

Lets Clean the Data

In [6]:
#remove unnamed coloumns from the website order report dataframe
order_report = order_report.drop(columns=['Unnamed: 3', 'Unnamed: 4'], errors='ignore')

#remove unnamed columns from SKU Master dataframe
sku_master =sku_master.drop(columns=['Unnamed: 2', 'Unnamed: 3' , 'Unnamed: 4'], errors='ignore')

#remove unnamed columns from pincode mapping dataframe
pincode_mapping = pincode_mapping.drop(columns=['Unnamed: 3', 'Unnamed: 4'], errors='ignore')



Now lets merge Order report and SKU Master datasets according to the common SKU column

In [7]:
#merge order report and SKU Master on SKU
merged_data = pd.merge(order_report, sku_master, on='SKU')
print(merged_data.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)
0     2001827036  8904223818706        1.0         127
1     2001821995  8904223818706        1.0         127
2     2001819252  8904223818706        1.0         127
3     2001816996  8904223818706        1.0         127
4     2001814580  8904223818706        1.0         127


The ‘ExternOrderNo’ is nothing but ‘Order Id’ in other datasets. Let’s rename this column:

In [12]:
#Rename the "ExternOrderNo" column to "OrderID" in the merge dataframe
merged_data = merged_data.rename(columns={'ExternOrderNo': 'Order ID'})

Now let’s merge the courier invoice and pincode mapping dataset:

In [8]:
#remove duplicates from pincode mapping
abc_courier = pincode_mapping.drop_duplicates(subset=['Customer Pincode'])

#We then select specific columns (“Order ID”, “Customer Pincode”, “Type of Shipment”)
# from the courier_invoice dataset and create a new DataFrame called “courier_abc” to store this subset of data.
courier_abc = courier_invoice[['Order ID', 'Customer Pincode', 'Type of Shipment']]

#We then merge the ‘courier_abc’ DataFrame with the ‘abc_courier’ DataFrame based on the ‘Customer Pincode’ column. This merge operation helps us associate customer pin codes with their respective orders and shipping types.
#The resulting DataFrame is named ‘pincodes’.
pincodes = courier_abc.merge(abc_courier, on= 'Customer Pincode')
print(pincodes.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


Now let’s merge the pin codes with the main dataframe:

In [13]:
merged2 = merged_data.merge(pincodes, on = 'Order ID')
print(merged2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Zone  
0  Forward charges             121003    e  
1  Forward charges             121003    e  
2  Forward charges             121003    e  
3  Forward charges             121003    e  
4  Forward charges             121003    e  


Now let’s calculate the weight in kilograms by dividing the ‘Weight (g)’ column in the ‘merged2’ DataFrame by 1000

In [22]:
merged2['Weights (Kgs)'] = merged2['Weight (g)'] / 1000

In [25]:
#Now let’s calculate the weight slabs:
def weight_slab(weight):
    """

    Args:
      weight:

    Returns:

    """
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1.0
    else:
        return int(weight) + 0.5
merged2['Weight Slab (KG)'] = merged2['Weights (Kgs)'].apply(weight_slab)
courier_invoice['Weight Slab Charged by Courier Company']=(courier_invoice['Charged Weight']).apply(weight_slab)

Now let’s rename the columns to prepare the desired dataframe:

In [26]:
courier_invoice = courier_invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})
merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per ABC'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per ABC'})

Now let’s calculate the expected charges:

In [27]:
total_expected_charge = []

for _, row in merged2.iterrows():
    fwd_category = 'fwd_' + row['Delivery Zone As Per ABC']
    fwd_fixed = courier_company_rates.at[0, fwd_category + '_fixed']
    fwd_additional = courier_company_rates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone As Per ABC']
    rto_fixed = courier_company_rates.at[0, rto_category + '_fixed']
    rto_additional = courier_company_rates.at[0, rto_category + '_additional']

    weight_slab = row['Weight Slab As Per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)

merged2['Expected Charge as per ABC'] = total_expected_charge
print(merged2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per ABC  weight_kg  \
0  Forward charges             121003                        e      0.127   
1  Forward charges             121003                        e      0.150   
2  Forward charges             121003                        e      0.100   
3  Forward charges             121003                        e      0.165   
4  Forward charges             121003                        e      0.350   

   Weight_(kgs)  Weights (Kgs)  Weight Slab As Per ABC  \
0         0.127          0.127          

In [28]:
merged_output = merged2.merge(courier_invoice, on='Order ID')
print(merged_output.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode_x  \
0  2001827036  8904223818706        1.0         127              173213   
1  2001827036  8904223819093        1.0         150              173213   
2  2001827036  8904223819109        1.0         100              173213   
3  2001827036  8904223818430        1.0         165              173213   
4  2001827036  8904223819277        1.0         350              173213   

  Type of Shipment_x  Warehouse Pincode_x Delivery Zone As Per ABC  weight_kg  \
0    Forward charges               121003                        e      0.127   
1    Forward charges               121003                        e      0.150   
2    Forward charges               121003                        e      0.100   
3    Forward charges               121003                        e      0.165   
4    Forward charges               121003                        e      0.350   

   Weight_(kgs)  ...  Weight Slab As Per ABC  Expected Charge 